# Data Handling Co-simulation Notebook

This notebook showcases co-simulation with Sedaro by creating `ExternalState` blocks in the Data Handling Co-simulation Scenario to receive and send simulation state to a running scenario simulation. This notebook starts a simulation of the scenario and consumes state from the satellite agent to calculate the signal strength of a data interface and produces state for the agent to activate a data interface.

## Setup


### Notebook Variables


In [ ]:
SCENARIO_BRANCH_ID = ...

MIN_RECEIVER_POWER = ...  # [W]
WAVELENGTH = ...  # [m]
TX_GAIN = ...
RX_GAIN = ...

#### Important: Read Before Running

These notebooks may make changes to agent and scenario branches in your account. Ensure any changes to the target branches are saved prior to running any code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

These notebooks also require that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as these notebooks with the following format:

```json
{
  "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.


In [ ]:
import json

with open('../secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('../config.json', 'r') as file:
    config = json.load(file)

### Client Objects


In [ ]:
from sedaro import SedaroApiClient

client = SedaroApiClient(api_key=API_KEY, host=config['HOST'])
scenario = client.scenario(SCENARIO_BRANCH_ID)
simulation = scenario.simulation

### Scenario Information


In [ ]:
ground_agent_names_by_id = {agent.id: agent.name for agent in scenario.PeripheralGroundPoint.get_all()}

### Agent Template Blocks


In [ ]:
satellite = scenario.TemplatedAgent.get_first()
satellite_template = client.agent_template(satellite.templateRef)

data_interface = satellite_template.CooperativeLineOfSightTransmitInterface.get_first()

modem = satellite_template.Modem.get_first()
modem_load_state = modem.loadStates[0]
modem_load = modem_load_state.loads[0]

power_processor = satellite_template.PowerProcessor.get_first()
controller_power_rating = power_processor.topologyParams['outputPowerRating']

target_group = satellite_template.TargetGroup.get_first()

### External State Blocks


In [ ]:
scenario.delete_all_external_state_blocks()

In [ ]:
cdh_state_block = scenario.SpontaneousExternalState.create(
    consumed=[{f"block.{target_group.id}.activeTarget": [
        {"static": "id"},
        {"agentElevation.as": "Angle.deg"},
        "lineOfSight",
        {"range.as": "Distance.m"},
        {"rangeRate.as": "Speed.km/s"},
    ]}],
    produced=[{f"block.{data_interface.id}": "isActive"}],
    engineIndex=1,
    agents=[satellite.id],
)

In [ ]:
power_state_block = scenario.SpontaneousExternalState.create(
    consumed=[{"PowerProcessor": {"prev": {"outputPowers": {"index": "total"}}}}],
    produced=[
        {f"block.{modem.id}": "activeLoadState"},
        {f"block.{modem_load.id}": "isActive"},
        {f"block.{modem_load.id}": "powerConsumed"},
    ],
    engineIndex=2,
    agents=[satellite.id],
)

### Interface Functions


In [ ]:
from math import pi


def friis_power_ratio(
        distance: float,
        wavelength: float,
        transmitter_gain: float,
        receiver_gain: float,
) -> float:
    """Calculate the received power to transmitted power ratio using the Friis transmission equation."""
    return transmitter_gain * receiver_gain * (wavelength / (4 * pi * distance))**2


def interface_state(
        elevation_angle: float,
        line_of_sight: bool,
        range_: float,
        total_power_consumed: float,
        controller_power_rating: float,
        min_receiver_power: float,
        wavelength: float,
        transmitter_gain: float,
        receiver_gain: float,
) -> tuple[bool, float]:
    """Determine if the data interface should be active and how much power the modem should consume based on target state."""
    available_power = controller_power_rating - total_power_consumed
    necessary_transmission_power = min_receiver_power / \
        friis_power_ratio(range_, wavelength, transmitter_gain, receiver_gain)
    if line_of_sight \
            and elevation_angle > 10 \
            and necessary_transmission_power < available_power:
        is_active = True
        modem_power = necessary_transmission_power
    else:
        is_active = False
        modem_power = 0.

    return is_active, modem_power

## Simulation


In [ ]:
from IPython.display import clear_output

with scenario.simulation.start(wait=True) as simulation_handle:
    while simulation_handle.status()['status'] == 'RUNNING':
        # consume and unpack state from the simulation
        cdh_state = simulation_handle.consume(agent_id=satellite.id, external_state_id=cdh_state_block.id)
        (
            active_target_id,
            elevation_angle,
            line_of_sight,
            range_,
            range_rate
        ) = cdh_state[0]
        power_state = simulation_handle.consume(agent_id=satellite.id, external_state_id=power_state_block.id)
        (
            total_power_consumed,
        ) = power_state[0]

        # calculate state for the interface and modem
        is_interface_active, modem_power_consumed = interface_state(
            elevation_angle,
            line_of_sight,
            range_,
            total_power_consumed,
            controller_power_rating,
            MIN_RECEIVER_POWER,
            WAVELENGTH,
            TX_GAIN,
            RX_GAIN
        )

        # produce state to the simulation
        simulation_handle.produce(agent_id=satellite.id,
                                  external_state_id=cdh_state_block.id,
                                  values=(is_interface_active,))
        simulation_handle.produce(agent_id=satellite.id,
                                  external_state_id=power_state_block.id,
                                  values=(
                                      modem_load_state.id if modem_power_consumed else None,
                                      True if modem_power_consumed else False,
                                      modem_power_consumed
                                  ))

        # print simulation state
        clear_output(wait=True)
        print("\n".join([
            f"Active Target:\t\t\t\t{ground_agent_names_by_id[active_target_id]}",
            f"Target to Satellite Elevation Angle:\t{elevation_angle:.6f}\t[deg]",
            f"Line of Sight:\t\t\t\t{line_of_sight}",
            f"Range to Target:\t\t\t{range_ / 1000:.6f}\t[km]",
            f"Range Rate to Target:\t\t\t{range_rate:.6f}\t[km/s]",
            f"Total Agent Power Consumed:\t\t{total_power_consumed:.6f}\t[W]",
            f"Data Interface Active:\t\t\t{is_interface_active}",
            f"Modem Power Consumed:\t\t\t{modem_power_consumed:.6f}\t[W]",
        ]), flush=True)